In [3]:
import requests
# This is about getting the data[HTML page] of websites
topic_url='https://github.com/topics'
response=requests.get(topic_url)
print(response.status_code)

200


In [5]:
page_content=response.text
# print(page_content)
page_content[:1000]

with open("webpage.html","w+",encoding="utf-8") as f:
    f.write(page_content)
#print(page_content)

In [6]:
#After getting the data we have to parse[Structured way] using beautiful soup

from bs4 import BeautifulSoup
doc=BeautifulSoup(page_content,"html.parser")
#print(doc)


In [4]:
#To find how many topics present in the page[p tags]
# p_tags=doc.find_all('p')
# len(p_tags)


In [7]:
selection_class="f3 lh-condensed mb-0 mt-1 Link--primary"
p_tags=doc.find_all("p",{"class":selection_class})
len(p_tags)

30

In [8]:
#Topic discrpitions

topic_discrption="f5 color-fg-muted mb-0 mt-1"
topic_discrption_tags=doc.find_all("p",{"class":topic_discrption})
len(topic_discrption_tags)
# topic_discrption_tags[:5]

30

In [9]:
topic_link_tags=doc.find_all('a',{"class":'no-underline flex-grow-0'})
len(topic_link_tags)
#print(topic_link_tags)
topic0_url="https://github.com"+topic_link_tags[0]['href']
print(topic0_url)

https://github.com/topics/3d


In [10]:
#getting whole topic lists

topic_lst=[]
for tags in p_tags:
    topic_lst.append(tags.text)
print(topic_lst[:5])

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']


In [11]:
#topic descriptions
topic_desc=[]
for tags in topic_discrption_tags:
    topic_desc.append(tags.text)
topic_desc[:3]

['\n          3D modeling is the process of virtually developing the surface and structure of a 3D object.\n        ',
 '\n          Ajax is a technique for creating interactive web applications.\n        ',
 '\n          Algorithms are self-contained sequences that carry out a variety of tasks.\n        ']

In [52]:
topic_url=[]
base_url='https://github.com'
for tags in topic_link_tags:
    topic_url.append(base_url+tags['href'])
topic_url[5]

'https://github.com/topics/angular'

In [29]:
! pip install pandas

     --------------------------------------- 10.5/10.5 MB 44.8 kB/s eta 0:00:00
     --------------------------------------- 14.6/14.6 MB 30.8 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# To create CSV files pandas has to download
import pandas as pd

In [14]:
# we are arranging the topictitles,topic descriptions and links in one row
# dictonaries is the best way to do 
topics_dict={
    'titles':p_tags,
     'descriptions':topic_discrption_tags,
     'urls':topic_url
}
# print(topics_dict)


In [15]:
#Here we are segretatting the data in the structural way using pandas
topics_df=pd.DataFrame(topics_dict)
# topics_desc

In [16]:
#creating CSV
topics_df.to_csv('topics.csv',index=None)
    

In [38]:
#getting info about each and every topic
#Topic page url
topic_page_url=topic_url[1]
print(topic_page_url)
response=requests.get(topic_page_url)
response.status_code


https://github.com/topics/ajax


200

In [18]:
topic_doc=BeautifulSoup(response.text,'html.parser')
# topic_doc

In [19]:
# To get the No.of user and repo tags
h3_selection_class="f3 color-fg-muted text-normal lh-condensed"
repo_tags=topic_doc.find_all("h3",{"class":h3_selection_class})
len(repo_tags)
#repo_tags[0]

20

In [20]:
#user name
a_tags=repo_tags[0].find_all('a')
a_tags[0].text.strip()

'mrdoob'

In [21]:
#Repo name
a_tags[1].text.strip()


'three.js'

In [22]:
# To get repo url for the topic[3D]
base_url="htts://github.com"
repo_url=base_url+ a_tags[1]['href']
print(repo_url)

htts://github.com/mrdoob/three.js


In [23]:
#Stars tags
# stars_calss="Counter js-social-count"
span_tag="Counter js-social-count"
stars_tag=topic_doc.find_all('span',{"class":span_tag})
len(stars_tag)
# print(stars_tag)

20

In [24]:
stars_tag[0].text.strip()

'84.7k'

In [25]:
#making a function which gives all stars counts
def parse_star_count (stars_tag):
    stars_tag=stars_tag.strip()
    if stars_tag[-1]=='k':
        return int(float(stars_tag[:-1])*1000)
    return int(stars_tag)
    

In [26]:
parse_star_count(stars_tag[0].text.strip())

84700

In [27]:
#making a function which give username,reponame,stars count and url
def get_repo_info(h3_selection_class,stars_tag):
    #returns all info about repository
    a_tags=h3_selection_class.find_all('a')
    username=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_url=base_url+a_tags[1]['href']
    stars=parse_star_count(stars_tag.text.strip())
    return username,repo_name,stars,repo_url
    

In [37]:
get_repo_info(repo_tags[1],stars_tag[1])


('libgdx', 'libgdx', 20400, 'htts://github.com/libgdx/libgdx')

In [29]:
#getting username,reponame,stars and links in structrued way for 3D
topic_repo_dict={
    "username":[],
    "repo_name":[],
    "stars":[],
    "repo_url":[]
    }
for i in range(len(repo_tags)):
    repo_info=get_repo_info(repo_tags[i],stars_tag[i])
    topic_repo_dict["username"].append(repo_info[0])
    topic_repo_dict["repo_name"].append(repo_info[1])
    topic_repo_dict["stars"].append(repo_info[2])
    topic_repo_dict["repo_url"].append(repo_info[3])
    
    

In [35]:
# topic_repo_dict

In [53]:
#By using pandas library we will give the data in structured way
topic_repo_df=pd.DataFrame(topic_repo_dict)
# topic_repo_df

In [50]:
#Defining functions:
def get_topic_page(topics_url):
    #download the page
    topic_url='https://github.com/topics'
    response=requests.get(topics_url)    
    #check sucessfull response
    if response.status_code !=200:
        raise Exception("failed to raise load page {}".format(topic_url))
    #parse beautiful soup
    doc=BeautifulSoup(page_content,"html.parser")
    return topic_doc

#making a function which give username,reponame,stars count and url
def get_repo_info(h3_selection_class,stars_tag):
    #returns all info about repository
    a_tags=h3_selection_class.find_all('a')
    username=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_url=base_url+a_tags[1]['href']
    stars=parse_star_count(stars_tag.text.strip())
    return username,repo_name,stars,repo_url
    

def get_topic_repos(topic_doc):
    #get h3 tags
    h3_selection_class="f3 color-fg-muted text-normal lh-condensed"
    repo_tags=topic_doc.find_all("h3",{"class":h3_selection_class})
    #get stars tags
    span_tag="Counter js-social-count"
    stars_tag=topic_doc.find_all('span',{"class":span_tag})
    
    topic_repo_dict={
    "username":[],
    "repo_name":[],
    "stars":[],
    "repo_url":[]
    }
    
    #get repo_info
    for i in range(len(repo_tags)):
        repo_info=get_repo_info(repo_tags[i],stars_tag[i])
        topic_repo_dict["username"].append(repo_info[0])
        topic_repo_dict["repo_name"].append(repo_info[1])
        topic_repo_dict["stars"].append(repo_info[2])
        topic_repo_dict["repo_url"].append(repo_info[3])
    return pd.DataFrame(topic_repo_dict)
    


    
    
    

In [43]:
url4=topic_url[4]

In [46]:
topic4_doc=get_topic_page(url4)


In [48]:
topic5_repo=get_topic_repos(topic4_doc)

In [56]:
# topic5_repo

In [59]:
# get_topic_repos(get_topic_page(topic_url[5]))